In [31]:
import duckdb
import pandas as pd

In [32]:
conn = duckdb.connect("data/analytics.duckdb")

In [33]:
sql_query = """
CREATE OR REPLACE VIEW master_list_items_engagement AS
SELECT 
    fi.event_item_id,
    fi.event_id,
    fi.item_id,
    fi.item_name,
    fi.item_list_name,
    fi.item_list_id,
    fi.item_category,
    fi.item_brand,
    fi.price,
    fi.total_price,
    fi.quantity,
    fi.position_in_list,
    fi.has_discount,
    fi.discount_amount,
    fi.in_stock,
    fe.event_timestamp,
    fe.user_id,
    fe.session_id,
    fe.event_date,
    fe.event_hour,
    fe.processed_at as processed_at_fact_events
FROM fact_event_items fi
INNER JOIN fact_events fe ON fi.event_id = fe.event_id
WHERE fi.item_list_name IS NOT NULL
"""

In [34]:
_ = conn.execute(sql_query)

In [35]:
dataframe = conn.execute("SELECT * FROM master_list_items_engagement").fetchdf()
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   event_item_id             311 non-null    object        
 1   event_id                  311 non-null    object        
 2   item_id                   311 non-null    object        
 3   item_name                 302 non-null    object        
 4   item_list_name            311 non-null    object        
 5   item_list_id              311 non-null    object        
 6   item_category             0 non-null      object        
 7   item_brand                0 non-null      object        
 8   price                     311 non-null    float64       
 9   total_price               304 non-null    float64       
 10  quantity                  311 non-null    int32         
 11  position_in_list          311 non-null    int32         
 12  has_discount          

In [36]:
import plotly.express as px

conteo_por_lista = dataframe['item_list_name'].value_counts().reset_index()
conteo_por_lista.columns = ['item_list_name', 'total_consultas']

fig = px.bar(conteo_por_lista, 
             x='total_consultas', 
             y='item_list_name', 
             orientation='h',
             title='Frecuencia de consultas por Item List Name',
             labels={'total_consultas': 'Número de consultas', 'item_list_name': 'Item List Name'})
fig.show()